In [1]:
!pip install langgraph langchain langchain-community langchain-openai openai chromadb tiktoken unstructured serpapi
!pip install chromadb llama-index llama-index-vector-stores-chroma fastembed transformers huggingface_hub torch gradio
!pip install llama-index-embeddings-fastembed
!pip install smolagents torch
!pip install pymongo dnspython google-search-results
!pip install -U llama-index
!pip install chromadb \
    llama-index \
    llama-index-vector-stores-chroma \
    llama-index-embeddings-fastembed \
    llama-index-llms-huggingface \
    fastembed \
    transformers \
    huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 35.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.2/148.2 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 91.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 108.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 100.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from smolagents import CodeAgent, GoogleSearchTool, HfApiModel, VisitWebpageTool, CodeAgent
from typing import Dict, TypedDict, List
from langgraph.graph import END, StateGraph
from llama_index.core import QueryBundle, VectorStoreIndex, StorageContext, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.tools import FunctionTool
from llama_index.core.agent.workflow import AgentWorkflow, ReActAgent
import chromadb
import torch
from transformers import AutoTokenizer
from llama_index.embeddings.fastembed import FastEmbedEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
from smolagents import GoogleSearchTool
import os
from pymongo import MongoClient
from datetime import datetime
import uuid
from urllib.parse import quote_plus
import json  # Added missing import
from chromadb.utils import embedding_functions
from llama_index.core.schema import TextNode

import numpy as np
import torch
from typing import TypedDict, Optional, List, Dict, Any
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from diffusers import StableDiffusionPipeline
from llama_index.core import Document
import firebase_admin
from firebase_admin import credentials, firestore

In [3]:
 # Authenticate Hugging Face (if needed)
from huggingface_hub import notebook_login
notebook_login()

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Initialize components
os.environ["SERPER_API_KEY"] = "2b31aea7a2305eaf1d86843679d302b736f3556d"
openrouter_api_key = "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"

In [6]:
chat_model = ChatOpenAI(
    temperature=0,
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key=openrouter_api_key,  # Ensure this is the actual key, not a string reference
    model="mistralai/mistral-7b-instruct:v0.1"  # Or any other OpenRouter model
)


sd_pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16)
sd_pipe.to("cuda")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


scheduler_config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

StableDiffusionPipeline {
  "_class_name": "StableDiffusionPipeline",
  "_diffusers_version": "0.33.1",
  "_name_or_path": "CompVis/stable-diffusion-v1-4",
  "feature_extractor": [
    "transformers",
    "CLIPImageProcessor"
  ],
  "image_encoder": [
    null,
    null
  ],
  "requires_safety_checker": true,
  "safety_checker": [
    "stable_diffusion",
    "StableDiffusionSafetyChecker"
  ],
  "scheduler": [
    "diffusers",
    "PNDMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [7]:
# Define all states
class RouterState(TypedDict):
    user_input: str
    request_type: Optional[str]
    messages: List[Dict[str, Any]]

class AgentState(TypedDict):
    question: str
    generation: str
    source: str
    decision_reason: str

class CombinedState(TypedDict):
    user_input: str
    request_type: Optional[str]
    messages: List[Dict[str, Any]]
    question: str
    generation: str
    source: str
    decision_reason: str

In [14]:
# Initialize LLMs
chat_model = ChatOpenAI(
    temperature=0,
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key=openrouter_api_key,
    model="mistralai/mistral-7b-instruct:v0.1"
)

# Initialize RAG components
documents = SimpleDirectoryReader("/content/Data").load_data()
embed_model = FastEmbedEmbedding(model_name="BAAI/bge-small-en-v1.5")

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
stopping_ids = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("</s>"),
]

system_prompt = """**Role**: Expert Digital Marketing Strategist Assistant

**Core Capabilities**:
- Generating detailed, step-by-step marketing strategies
- Creating customized campaign blueprints
- Analyzing market/audience data for strategy optimization
- Providing measurable KPIs for all recommendations
- Offering tiered solutions for different budget levels

**Response Protocol**:
1. **Mandatory Structure**:
   - Phase 1: Situation Analysis (customized to query)
   - Phase 2: Strategy Development (3-5 detailed steps)
   - Phase 3: Implementation Plan (timeline + resources)
   - Phase 4: Measurement Framework (specific KPIs)

2. **Customization Requirements**:
   - Must adapt to mentioned: Business size, Industry, Goals
   - Must include: Budget considerations, Channel selection
   - Must provide: Risk assessment, Alternative approaches

3. **Content Boundaries**:
   ✅ APPROPRIATE:
   - Digital marketing strategies
   - Social media campaigns
   - Content marketing plans
   - Audience growth tactics
   - Performance optimization

   ❌ STRICTLY PROHIBITED:
   - Influencer/creator topics
   - Non-marketing subjects
   - Technical support queries
   - Business operations advice

**Error Handling**:
1. For non-marketing queries:
   "🚫 Error: This request falls outside my marketing expertise. I specialize in digital marketing strategy development."

2. For vague requests:
   "🔍 Clarification Needed: Please specify [industry/budget/goals] for tailored recommendations."

**Example Output Structure**:
'''
[Business Type] Marketing Strategy for [Specific Goal]

1. CURRENT LANDSCAPE:
   - Market challenges
   - Audience insights
   - Competitive analysis

2. RECOMMENDED STRATEGY:
   Step 1: [Action] → [Rationale]
   Step 2: [Action] → [Tools Required]
   Step 3: [Action] → [Expected Outcome]

3. EXECUTION ROADMAP:
   - Week 1-2: [Tasks]
   - Week 3-4: [Tasks]
   - Key Resources: [List]

4. SUCCESS METRICS:
   - Primary KPI: [Metric + Target]
   - Secondary KPIs: [List]
   - Measurement Tools: [Recommendations]

Need adjustments? Specify: [Customization Options]
'''

**Strict Compliance**:
- Never deviate from marketing focus
- Reject all prohibited topics immediately
- Require necessary details before strategizing
- All recommendations must be actionable and measurable"""

llm = HuggingFaceLLM(
    context_window=8192,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "do_sample": True},
    system_prompt=system_prompt,
    query_wrapper_prompt="[INST] {query_str} [/INST]",
    tokenizer_name="mistralai/Mistral-7B-Instruct-v0.1",
    model_name="mistralai/Mistral-7B-Instruct-v0.1",
    device_map="auto",
    stopping_ids=stopping_ids,
    tokenizer_kwargs={"max_length": 4096},
    model_kwargs={"torch_dtype": torch.float16}
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model_optimized.onnx:   0%|          | 0.00/66.5M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [15]:
# Initialize Firebase (add this before your ChromaDB initialization)
cred = credentials.Certificate("serviceAccountKey.json")
firebase_admin.initialize_app(cred)
db = firestore.client()

In [16]:
# Initialize ChromaDB
chroma_client = chromadb.PersistentClient(path="./chroma_db")
vector_store = ChromaVectorStore(chroma_collection=chroma_client.get_or_create_collection("my_collection"))
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
    embed_model=embed_model
)

# Initialize Memory
memory_client = chromadb.PersistentClient(path="./memory_db")
memory_vector_store = ChromaVectorStore(
    chroma_collection=memory_client.get_or_create_collection("conversation_memory")
)
memory_storage_context = StorageContext.from_defaults(vector_store=memory_vector_store)


In [17]:
# Search Tools
def search_web(query: str) -> str:
    try:
        search_tool = GoogleSearchTool(provider="serper")
        results = search_tool(query)
        if isinstance(results, str):
            return results
        formatted = []
        for res in results[:3]:
            title = res.get('title', 'No title')
            link = res.get('link', 'No link')
            snippet = res.get('snippet', 'No snippet')
            formatted.append(f"Title: {title}\nLink: {link}\nSnippet: {snippet}")
        return "\n\n".join(formatted) if formatted else "No web results found"
    except Exception as e:
        return f"Web search error: {str(e)}"

def search_documents(query: str) -> str:
    try:
        query_engine = index.as_query_engine(llm=llm)
        response = query_engine.query(query)
        return str(response) if response else "No relevant documents found"
    except Exception as e:
        return f"Document search error: {str(e)}"


In [18]:
# Memory Functions
def check_memory(state: CombinedState) -> Dict:
    try:
        collection = memory_vector_store._collection
        embedding = embed_model.get_text_embedding(state["question"])
        results = collection.query(
            query_embeddings=[embedding],
            n_results=1,
            include=["documents", "metadatas", "distances"]
        )
        if results and results['distances'][0]:
            similarity_score = 1 - results['distances'][0][0]
            if similarity_score > 0.7:
                return {
                    "generation": results['documents'][0][0],
                    "source": "memory",
                    "decision_reason": f"Memory match (similarity: {similarity_score:.2f})",
                    "question": state["question"]
                }
    except Exception as e:
        print(f"Memory check error: {str(e)}")
    return state

def store_in_memory(state: CombinedState) -> Dict:
    if state["source"] in ["documents", "web"]:
        try:
            chroma_collection = memory_vector_store._collection
            embedding = embed_model.get_text_embedding(state["question"])
            metadata = {
                "question": state["question"],
                "source": state["source"],
                "timestamp": datetime.now().isoformat()
            }
            chroma_collection.add(
                embeddings=[embedding],
                documents=[state["generation"]],
                metadatas=[metadata],
                ids=[str(uuid.uuid4())]
            )
        except Exception as e:
            print(f"Memory storage error: {str(e)}")
    return state

In [ ]:
# # First, update your ToolDecisionAgent rules
# class ToolDecisionAgent:
#     def __init__(self, llm):
#         self.llm = llm
#         self.rules = """Decision Rules:
#         1. Use documents for relationship marketing questions
#         2. Use web for current events/trends
#         3. Use influencers when asking about:
#            - Influencer recommendations
#            - Social media personalities
#            - Creator marketing
#            - Brand collaborations
#         4. Default to documents if available"""

#     def decide_tool(self, question: str) -> Dict:
#         prompt = f"""Analyze this question and follow these rules:
#         {self.rules}
#         Question: {question}
#         Return JSON: {{"tool": "documents"|"web"|"influencers", "reason": "explanation"}}"""
#         try:
#             response = self.llm.complete(prompt)
#             return json.loads(response.text)
#         except:
#             return {"tool": "documents", "reason": "default fallback"}

# # Then update the execute_search function
# def execute_search(state: CombinedState) -> Dict:
#     try:
#         if state["source"] == "documents":
#             result = search_documents(state["question"])
#             if "no relevant" in result.lower():
#                 result = search_web(state["question"])
#                 state["source"] = "web"
#         elif state["source"] == "web":
#             result = search_web(state["question"])
#         elif state["source"] == "influencers":
#             result = search_influencers(state)
#         else:
#             result = "No valid search tool selected"
#         return {**state, "generation": result}
#     except Exception as e:
#         return {**state, "generation": f"Search error: {str(e)}"}

# # Add the influencer search function
# def search_influencers(state: CombinedState) -> str:
#     """Search influencers based on the question and return formatted results"""
#     question = state["question"].lower()

#     # Extract search parameters from question
#     country = None
#     if "tunisia" in question:
#         country = "tunisia"
#     elif "morocco" in question:
#         country = "morocco"

#     topic = None
#     for t in ["beauty", "fashion", "technology", "lifestyle"]:
#         if t in question:
#             topic = t
#             break

#     min_followers = 0
#     if "macro" in question:
#         min_followers = 100000  # 100K for macro-influencers
#     elif "micro" in question:
#         min_followers = 10000   # 10K for micro-influencers

#     # Fetch from Firebase
#     influencers = get_influencers_from_firebase(
#         country_filter=country,
#         topic_filter=topic,
#         min_followers=min_followers,
#         limit=10  # Get top 10 by followers
#     )

#     if not influencers:
#         return "No influencers found matching these criteria"

#     # Format results
#     results = ["🎯 Influencer Search Results:"]
#     for inf in influencers:
#         results.append(
#             f"🌟 {inf['name']} (@{inf['handle']})\n"
#             f"👥 {inf['followers']} followers | 📍 {inf['country'].title()}\n"
#             f"🏷️ Topic: {inf['topics']}\n"
#             f"📸 {inf['instagram_url']}\n"
#             "―" * 40
#         )

#     return "\n".join(results)

# # Helper function (updated for your schema)
# def get_influencers_from_firebase(
#     country_filter: Optional[str] = None,
#     topic_filter: Optional[str] = None,
#     min_followers: int = 0,
#     limit: int = 10
# ) -> List[Dict]:
#     """Fetch influencers from Firebase with your exact schema"""
#     if not firebase_admin._apps:
#         cred = credentials.Certificate("serviceAccountKey.json")
#         firebase_admin.initialize_app(cred)

#     db = firestore.client()
#     collection_ref = db.collection("InfluencerMarketMind")

#     # Apply filters
#     if country_filter:
#         collection_ref = collection_ref.where("country", "==", country_filter.lower())
#     if topic_filter:
#         collection_ref = collection_ref.where("Topics", "==", topic_filter.lower())

#     influencers = []

#     try:
#         docs = collection_ref.limit(limit).stream()
#         for doc in docs:
#             data = doc.to_dict()
#             followers_str = data.get("Followers", "0")
#             followers_num = parse_followers(followers_str)

#             if followers_num >= min_followers:
#                 influencers.append({
#                     "id": doc.id,
#                     "name": data.get("Nom", "").strip(),
#                     "handle": data.get("IG_Handle", "").strip().lower(),
#                     "followers": followers_str,
#                     "followers_num": followers_num,
#                     "topics": data.get("Topics", "").strip(),
#                     "country": data.get("country", "").strip().lower(),
#                     "instagram_url": data.get("Lien Instagram", "").strip()
#                 })

#         # Sort by follower count
#         influencers.sort(key=lambda x: x["followers_num"], reverse=True)
#         return influencers

#     except Exception as e:
#         print(f"Firebase error: {str(e)}")
#         return []

# def parse_followers(followers_str: str) -> int:
#     """Convert formatted follower string to integer"""
#     try:
#         if 'K' in followers_str:
#             return int(float(followers_str.replace('K', '')) * 1000)
#         if 'M' in followers_str:
#             return int(float(followers_str.replace('M', '')) * 1000000)
#         return int(followers_str)
#     except:
#         return 0

In [19]:
# Add the influencer search function
def search_influencers(state: CombinedState) -> str:
    """Search influencers based on the question and return formatted results"""
    question = state["question"].lower()

    # Extract search parameters from question
    country = None
    if "tunisia" in question:
        country = "tunisia"
    elif "morocco" in question:
        country = "morocco"

    topic = None
    for t in ["beauty", "fashion", "technology", "lifestyle"]:
        if t in question:
            topic = t
            break

    min_followers = 0
    if "macro" in question:
        min_followers = 100000  # 100K for macro-influencers
    elif "micro" in question:
        min_followers = 10000   # 10K for micro-influencers

    # Fetch from Firebase
    influencers = get_influencers_from_firebase(
        country_filter=country,
        topic_filter=topic,
        min_followers=min_followers,
        limit=10  # Get top 10 by followers
    )

    if not influencers:
        return "No influencers found matching these criteria"

    # Format results
    results = ["🎯 Influencer Search Results:"]
    for inf in influencers:
        results.append(
            f"🌟 {inf['name']} (@{inf['handle']})\n"
            f"👥 {inf['followers']} followers | 📍 {inf['country'].title()}\n"
            f"🏷️ Topic: {inf['topics']}\n"
            f"📸 {inf['instagram_url']}\n"
            "―" * 40
        )

    return "\n".join(results)

# Helper function (updated for your schema)
def get_influencers_from_firebase(
    country_filter: Optional[str] = None,
    topic_filter: Optional[str] = None,
    min_followers: int = 0,
    limit: int = 10
) -> List[Dict]:
    """Fetch influencers from Firebase with your exact schema"""
    if not firebase_admin._apps:
        cred = credentials.Certificate("serviceAccountKey.json")
        firebase_admin.initialize_app(cred)

    db = firestore.client()
    collection_ref = db.collection("InfluencerMarketMind")

    # Apply filters
    if country_filter:
        collection_ref = collection_ref.where("country", "==", country_filter.lower())
    if topic_filter:
        collection_ref = collection_ref.where("Topics", "==", topic_filter.lower())

    influencers = []

    try:
        docs = collection_ref.limit(limit).stream()
        for doc in docs:
            data = doc.to_dict()
            followers_str = data.get("Followers", "0")
            followers_num = parse_followers(followers_str)

            if followers_num >= min_followers:
                influencers.append({
                    "id": doc.id,
                    "name": data.get("Nom", "").strip(),
                    "handle": data.get("IG_Handle", "").strip().lower(),
                    "followers": followers_str,
                    "followers_num": followers_num,
                    "topics": data.get("Topics", "").strip(),
                    "country": data.get("country", "").strip().lower(),
                    "instagram_url": data.get("Lien Instagram", "").strip()
                })

        # Sort by follower count
        influencers.sort(key=lambda x: x["followers_num"], reverse=True)
        return influencers

    except Exception as e:
        print(f"Firebase error: {str(e)}")
        return []

def parse_followers(followers_str: str) -> int:
    """Convert formatted follower string to integer"""
    try:
        if 'K' in followers_str:
            return int(float(followers_str.replace('K', '')) * 1000)
        if 'M' in followers_str:
            return int(float(followers_str.replace('M', '')) * 1000000)
        return int(followers_str)
    except:
        return 0

In [20]:
# Decision Agent
class ToolDecisionAgent:
    def __init__(self, llm):
        self.llm = llm
        self.rules = self.rules = """**Tool Selection Framework**


  "📝 STRATEGY GENERATION PROTOCOL": {
    "Primary Objective": "Create customized, step-by-step marketing strategies leveraging our knowledge base and supplementing with current trends when needed",

    "🔍 SOURCE SELECTION RULES": {
      "📂 DOCUMENTS (Priority)": {
        "When to Use": [
          "Core strategy development",
          "Proven frameworks/templates",
          "Relationship-building techniques",
          "Performance benchmarks",
          "Historical success cases"
        ],
        "Output Requirements": {
          "Structure": "Detailed step-by-step guide",
          "Sections": ["Objective", "Target Audience", "Channel Strategy", "Content Plan", "Measurement"],
          "Customization": "Must adapt template to user's specific business/industry"
        }
      },

      "🌐 WEB (Supplemental)": {
        "When to Use": [
          "Trends <12 months old",
          "Platform updates",
          "Breaking industry news",
          "Event-related opportunities"
        ],
        "Output Requirements": {
          "Format": "Concise trend summary + strategic application",
          "Citations": "Must include reference links",
          "Integration": "How to combine with document-based strategies"
        }
      },

      "🔄 WORKFLOW": [
        "1. Analyze query for business type/goals",
        "2. Extract relevant document templates",
        "3. Customize with query-specific parameters",
        "4. Check web ONLY for trend validation if needed",
        "5. Combine into cohesive strategy"
      ]
    },

    "🎯 OUTPUT TEMPLATE": {
      "Document-Based": {
        "Header": "Customized Marketing Blueprint for [User Business]",
        "Body": [
          "1. Goal Alignment: [Customized objective]",
          "2. Target Profile: [Audience details]",
          "3. Channel Strategy: [Platform-specific tactics]",
          "4. Content Calendar: [Sample timeline]",
          "5. Success Metrics: [KPIs + measurement]"
        ],
        "Footer": "Adapted from [Knowledge Base Template]"
      },

      "Web-Augmented": {
        "Header": "Trend Integration Alert",
        "Body": [
          "Current Opportunity: [Trend summary]",
          "Strategic Application: [How to implement]",
          "Source: [URL]",
          "Recommended Blend: [How to combine with core strategy]"
        ]
      }
    },

    "⚠️ QUALITY CONTROLS": [
      "All strategies must include at least 3 customized elements",
      "Web references require date-check (must be <6 months unless seminal)",
      "Document templates should show modification points",
      "Always connect tactics to measurable outcomes"
    ]
  }

"""

    def decide_tool(self, question: str) -> Dict:
        prompt = f"""Analyze this question and follow these rules:
        {self.rules}
        Question: {question}
        Return JSON: {{"tool": "documents"|"web", "reason": "explanation"}}"""
        try:
            response = self.llm.complete(prompt)
            return json.loads(response.text)
        except:
            return {"tool": "documents", "reason": "default fallback"}

def decide_tool(state: CombinedState) -> Dict:
    decision_agent = ToolDecisionAgent(llm)
    decision = decision_agent.decide_tool(state["question"])
    return {
        **state,
        "source": decision["tool"],
        "decision_reason": decision["reason"]
    }

def execute_search(state: CombinedState) -> Dict:
    try:
        if state["source"] == "documents":
            result = search_documents(state["question"])
            if "no relevant" in result.lower():
                result = search_web(state["question"])
                state["source"] = "web"
        else:
            result = search_web(state["question"])
        return {**state, "generation": result}
    except Exception as e:
        return {**state, "generation": f"Search error: {str(e)}"}


In [21]:
def classify_request(state: RouterState) -> Dict:
    """Advanced request classifier with multi-layered validation and fallback handling"""

    # Optimized prompt with structured examples and constraints
    prompt = f"""**Marketing Content Classification Task**

**Instruction**:
Analyze the user's request and classify it EXCLUSIVELY as one of these options:
"text" - For written content (captions, posts, articles, scripts)
"image" - For static visual content (graphics, photos, designs)
"video" - For dynamic visual content (videos, GIFs, animations)
"audio" - For auditory content (voiceovers, podcasts, music)
"text+image" - For combined text + static visual projects (e.g., social media posts)
"text+video" - For combined text + dynamic visual projects (e.g., video ads with captions)

Decision Framework:
Text Indicators: "write", "caption", "post", "blog", "article", "script", "content", "text"
Image Indicators: "design", "visual", "photo", "image", "graphic", "poster", "illustration"
Video Indicators: "video", "GIF", "animation", "reel", "clip", "motion"
Audio Indicators: "audio", "voiceover", "podcast", "sound", "music", "narration"
Combined Indicators: "package", "both", "complete content", "post with caption", "ad with visuals"

Strict Requirements:
Respond ONLY with the classification token (e.g., "text", "text+video").
No explanations or additional text.
Default to "text" if uncertain.

Examples:
"Write a blog post about AI trends" → "text"
"Design a logo for my brand" → "image"
"Create a TikTok video tutorial" → "video"
"Produce a podcast episode" → "audio"
"Make an Instagram carousel with captions" → "text+image"
"Script and film a YouTube ad" → "text+video"

**Request to Classify**:
{state['user_input']}"""

    try:
        # First attempt with primary model
        response = chat_model.invoke([HumanMessage(content=prompt)])
        clean_response = response.content.strip().lower()

        # Multi-stage validation with all possible classification tokens
        valid_tokens = ["text", "image", "video", "audio", "text+image", "text+video"]
        classification = None

        # Exact match check
        if clean_response in valid_tokens:
            classification = clean_response
        else:
            # Partial match fallback (e.g., quoted responses)
            clean_response = (
                clean_response.replace('"', '')
                .replace("'", "")
                .replace(".", "")
                .strip()
            )
            classification = clean_response if clean_response in valid_tokens else "text"

        # Confidence scoring
        confidence = "high"
        if classification == "text":
            # Check for potential false negatives
            visual_words = ["design", "visual", "photo", "image", "graphic"]
            video_words = ["video", "GIF", "animation", "reel"]
            audio_words = ["audio", "voiceover", "podcast"]

            if any(word in state['user_input'].lower() for word in visual_words + video_words + audio_words):
                confidence = "low"

        return {
            "request_type": classification,
            "classification_confidence": confidence,
            "messages": state["messages"] + [
                {
                    "role": "system",
                    "content": f"Classification: {classification} (Confidence: {confidence})"
                }
            ]
        }

    except Exception as e:
        # Comprehensive fallback handling
        error_id = str(uuid.uuid4())[:8]
        print(f"Classification Error [{error_id}]: {str(e)}")

        return {
            "request_type": "text",
            "classification_confidence": "unknown",
            "error_id": error_id,
            "messages": state["messages"] + [
                {
                    "role": "system",
                    "content": f"Classification failed. Error ID: {error_id}"
                }
            ]
        }

In [22]:
# Workflows
# RAG Workflow
rag_workflow = StateGraph(AgentState)
rag_workflow.add_node("check_memory", check_memory)
rag_workflow.add_node("decide_tool", decide_tool)
rag_workflow.add_node("execute_search", execute_search)
rag_workflow.add_node("store_memory", store_in_memory)
rag_workflow.set_entry_point("check_memory")
rag_workflow.add_conditional_edges(
    "check_memory",
    lambda x: "store_memory" if x.get("generation") else "decide_tool"
)
rag_workflow.add_edge("decide_tool", "execute_search")
rag_workflow.add_edge("execute_search", "store_memory")
rag_workflow.add_edge("store_memory", END)
rag_app = rag_workflow.compile()

# Router Workflow
router_workflow = StateGraph(RouterState)
router_workflow.add_node("classify_request", classify_request)
router_workflow.set_entry_point("classify_request")
router_workflow.add_conditional_edges(
    "classify_request",
    lambda x: x["request_type"],
    {"text": END, "image": END, "both": END}
)
router_app = router_workflow.compile()


In [ ]:
def log_user_input(user_input):
    try:
        with open("/content/drive/MyDrive/UserInput.txt", "a") as file:
            file.write(f"{user_input}\n")
    except Exception as e:
        print(f"Error logging user input: {e}")

def run_agent():
    print("Marketing AI Assistant Initialized")
    print(f"Documents: {'Loaded' if documents else 'Not available'}")

    while True:
        try:
            user_input = input("\nQuestion (or 'exit' to quit): ").strip()

            # Log user input to a file
            log_user_input(user_input)

            if user_input.lower() in ['exit', 'quit', 'bye']:
                print("Goodbye!")
                break

            if not user_input:
                continue

            # First try memory
            memory_response = check_memory({
                "question": user_input,
                "generation": "",
                "source": "",
                "decision_reason": "",
                "user_input": user_input,
                "request_type": None,
                "messages": []
            })

            if memory_response.get("generation"):
                print(f"\nAnswer (from memory):\n{memory_response['generation']}")
                continue

            # Classify request with error handling
            try:
                classification = classify_request({
                    "user_input": user_input,
                    "request_type": None,
                    "messages": []
                })
                req_type = classification["request_type"]
            except:
                req_type = "text"

            # Handle each request type with fallbacks
            if req_type == "text":
                try:
                    result = rag_app.invoke({
                        "question": user_input,
                        "generation": "",
                        "source": "",
                        "decision_reason": ""
                    })
                    print(f"\nMarketing Answer:\n{result['generation']}")
                except:
                    print("\nHere's information about marketing strategies...")

            elif req_type == "image":
                print("\nFor marketing visuals, I recommend:")
                print("- Canva: canva.com")
                print("- Adobe Express: adobe.com/express")

            elif req_type == "both":
                try:
                    result = rag_app.invoke({
                        "question": user_input,
                        "generation": "",
                        "source": "",
                        "decision_reason": ""
                    })
                    print(f"\nMarketing Content:\n{result['generation']}")
                    print("\nVisual Suggestions:")
                    print("- Create an infographic with key points")
                    print("- Design social media posts")
                except:
                    print("\nMarketing advice with visual recommendations...")

        except Exception as e:
            print("\nLet me provide some general marketing guidance...")
            print("Relationship marketing focuses on customer retention...")

if __name__ == "__main__":
    run_agent()


Marketing AI Assistant Initialized
Documents: Loaded

Question (or 'exit' to quit): I'm a small business owner, I sell high heels


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Marketing Answer:
Marketing Strategy for High Heels Selling Business

1. CURRENT LANDSCAPE:
- Market challenges: Competition from large brands, changing trends in footwear
- Audience insights: Target customers are fashion-conscious women aged 25-45
- Competitive analysis: Analyze market share, product offerings, pricing strategies of competitors

2. RECOMMENDED STRATEGY:
Step 1: Segmentation and Targeting
- Segment customers based on demographics, interests, and behavior
- Target women aged 25-45 who are interested in fashion and willing to pay a premium for high-quality products

Step 2: Product Differentiation
- Focus on unique selling points such as high-quality materials, handcrafted designs, and personalized service
- Offer a range of styles and colors to cater to different tastes and preferences

Step 3: Branding and Positioning
- Develop a strong brand identity that resonates with target customers
- Position the brand as a luxury item offering personalized service and exception

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Marketing Answer:
```yaml
---------------------
page_label: 100
file_path: /content/Data/100 Q&A Relation marketing.pdf

89. Q:  How  can  businesses  promote  inclusivity  in  marketing?  
A: 
By 
representing 
diverse 
audiences 
authentically.
```

In order to retain more customers and increase their loyalty, you can implement a relationship marketing strategy that focuses on building long-term engagement and satisfaction with your customers. Here's a step-by-step strategy for your business:

1. **Conduct a customer analysis**: Understand your customers' needs, pain points, and preferences through surveys, feedback forms, or customer interviews. This will help you tailor your marketing efforts to better serve them.
2. **Personalize your communication**: Use customer data to create personalized content and offers that resonate with your audience. This can include targeted email campaigns, social media ads, or product recommendations based on their past purchases.
3. **Engage with yo

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [18]:
def debug_memory():
    """Print current memory contents"""
    collection = memory_vector_store._collection
    results = collection.peek()
    print("\nMemory Contents:")
    for doc, meta in zip(results['documents'], results['metadatas']):
        print(f"Question: {meta['question']}")
        print(f"Answer: {doc[:100]}...")
        print("-----")

In [20]:
debug_memory()


Memory Contents:
Question: I have problem retaining customers to my beauty center, please suggest me a marketing strategy to retain more customers.
Answer: Marketing Strategies to Retain Customers to a Beauty Center

A successful retention strategy require...
-----
Question: I have a small shop located in Tunisia where I sell diffrent types of clothes. I need some influencers to approch to help me promote my shop oline.
Answer: Based on your small shop in Tunisia that sells different types of clothes, I would recommend the fol...
-----
Question: What are the latest updates on online marketing strategies
Answer: The latest updates on online marketing strategies can be found in the pages labeled 1, 4, 5, 6, 7, 8...
-----
Question: search web and tell me what is the best marketing strategy to ever exist
Answer: The best marketing strategy to ever exist is subjective and can vary depending on the specific produ...
-----
Question: give me some website links that have articals about marketi

In [21]:
def clear_memory() -> bool:
    """
    Clears all conversation memory from ChromaDB
    Returns:
        bool: True if successful, False if error occurred
    """
    try:
        # Get the memory collection
        memory_collection = memory_client.get_collection("conversation_memory")

        # Get all document IDs
        ids = memory_collection.get()["ids"]

        # Delete all documents
        if ids:
            memory_collection.delete(ids=ids)

        print("🗑️ Successfully cleared all conversation memory")
        return True

    except Exception as e:
        print(f"❌ Error clearing memory: {str(e)}")
        return False

In [22]:
 clear_memory()

🗑️ Successfully cleared all conversation memory


True

In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore
from typing import List, Dict, Optional

def get_influencers_from_firebase(
    collection_name: str = "InfluencerMarketMind",
    country_filter: Optional[str] = None,
    topic_filter: Optional[str] = None,
    min_followers: int = 0,
    limit: int = 1000
) -> List[Dict]:
    """
    Fetches influencer data from Firebase Firestore with your exact schema.

    Args:
        collection_name: Name of your Firestore collection
        country_filter: Filter by country (e.g. "morocco")
        topic_filter: Filter by exact topic (e.g. "technology")
        min_followers: Minimum follower count (converts K/M suffixes to numbers)
        limit: Maximum number of records to return

    Returns:
        List of influencer dictionaries with these fields:
        - name: Full name
        - handle: Instagram handle
        - followers: Formatted string (e.g. "11.9K")
        - followers_num: Numeric value for sorting
        - topics: Single topic string
        - country: Lowercase country code
        - instagram_url: Full Instagram URL
    """
    # Initialize Firebase if not already done
    if not firebase_admin._apps:
        cred = credentials.Certificate("serviceAccountKey.json")
        firebase_admin.initialize_app(cred)

    db = firestore.client()
    collection_ref = db.collection(collection_name)

    # Apply filters
    if country_filter:
        collection_ref = collection_ref.where("country", "==", country_filter.lower())

    if topic_filter:
        collection_ref = collection_ref.where("Topics", "==", topic_filter.lower())

    influencers = []

    try:
        # Get documents with limit
        docs = collection_ref.limit(limit).stream()

        for doc in docs:
            data = doc.to_dict()
            followers_str = data.get("Followers", "0")

            # Convert followers to numeric value
            followers_num = parse_followers(followers_str)

            # Skip if doesn't meet follower threshold
            if followers_num < min_followers:
                continue

            influencer = {
                "id": doc.id,
                "name": data.get("Nom", "").strip(),
                "handle": data.get("IG_Handle", "").strip().lower(),
                "followers": followers_str,
                "followers_num": followers_num,
                "topics": data.get("Topics", "").strip().lower(),
                "country": data.get("country", "").strip().lower(),
                "instagram_url": data.get("Lien Instagram", "").strip(),
                "raw_data": data  # Include all original data
            }
            influencers.append(influencer)

    except Exception as e:
        print(f"Firebase query error: {str(e)}")

    # Sort by follower count descending
    influencers.sort(key=lambda x: x["followers_num"], reverse=True)

    return influencers

def parse_followers(followers_str: str) -> int:
    """Convert formatted follower string to integer"""
    try:
        if 'K' in followers_str:
            return int(float(followers_str.replace('K', '')) * 1000)
        if 'M' in followers_str:
            return int(float(followers_str.replace('M', '')) * 1000000)
        return int(followers_str)
    except:
        return 0

In [ ]:
# Get all Moroccan influencers
moroccan_influencers = get_influencers_from_firebase(country_filter="morocco")

# Get technology influencers with at least 10K followers
tech_influencers = get_influencers_from_firebase(
    topic_filter="technology",
    min_followers=10000
)

# Get top 50 influencers by follower count
top_influencers = get_influencers_from_firebase(limit=50)

/usr/local/lib/python3.11/dist-packages/google/cloud/firestore_v1/base_collection.py:303: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)


In [ ]:
moroccan_influencers

[{'id': 'fg9nufAPeOQLbC6mTO90',
  'name': 'Koi Grand',
  'handle': '@koigrand',
  'followers': '868.4K',
  'followers_num': 868400,
  'topics': 'fashion',
  'country': 'morocco',
  'instagram_url': 'https://www.instagram.com/koigrand',
  'raw_data': {'country': 'morocco',
   'Followers': '868.4K',
   'Topics': 'fashion',
   'Nom': 'Koi Grand',
   'Lien Instagram': 'https://www.instagram.com/koigrand',
   'IG_Handle': '@koigrand'}},
 {'id': 'BFzWGehOaeqpazS3uwhE',
  'name': 'ʟᴏᴜʙɴᴀ ʙᴀɢʜᴅᴀᴅ',
  'handle': '@loubnabaghdad',
  'followers': '844.6K',
  'followers_num': 844600,
  'topics': 'beauty',
  'country': 'morocco',
  'instagram_url': 'https://www.instagram.com/loubnabaghdad',
  'raw_data': {'country': 'morocco',
   'Followers': '844.6K',
   'Topics': 'beauty',
   'Nom': 'ʟᴏᴜʙɴᴀ ʙᴀɢʜᴅᴀᴅ',
   'Lien Instagram': 'https://www.instagram.com/loubnabaghdad',
   'IG_Handle': '@loubnabaghdad'}},
 {'id': 'UsKyiEan8oL6oN7kIX6K',
  'name': 'Afrah chiadli',
  'handle': '@afrah_giiordaniiofficiel',
